In [1]:
import os
import pandas as pd

from openai import OpenAI
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers

In [2]:
# .env 파일에서 환경 변수 로드
load_dotenv()

#openai api key
api_key = os.getenv("OPENAI_API_KEY")

#elasticsearch password
es_pw = os.getenv('ES_PW')

In [3]:
# 정책, 용어 파일 로드
policy_data = pd.read_excel('origin_policy_chunk.xlsx')
words_data = pd.read_excel('origin_word_chunk.xlsx')

In [4]:
policy_data.head()

,rnum,정책
0,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
1,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
2,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
3,2,지역: 은평구\n분야: 일자리\n정책명: 청년 창업점포 운영\n정책 설명: 청년 창...
4,2,지역: 은평구\n분야: 일자리\n정책명: 청년 창업점포 운영\n정책 설명: 청년 창...


In [5]:
words_data.head()

,rnum,용어
0,1,디지털 드로잉은 컴퓨터나 태블릿과 같은 디지털 기기를 사용하여 그림을 그리는 것을 ...
1,2,만 나이는 사람이 실제로 살아온 정확한 연수를 기준으로 계산하는 나이입니다. 한국에...
2,3,"비영리단체는 사회적, 교육적, 자선적, 문화적 또는 과학적 목적을 추구하기 위해 설..."
3,4,협동조합은 공동의 목표를 가진 회원들이 자발적으로 모여 운영하는 비즈니스 조직입니다...
4,5,새싹점포 청년상인은 전통시장이나 상점가에서 새롭게 사업을 시작하는 젊은 상인들을 의...


In [6]:
#임베딩 설정

# OpenAI API 키 설정
openai = OpenAI()
openai.api_key = api_key

In [9]:
# 텍스트 임베딩 함수
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return openai.embeddings.create(input = [text], model=model).data[0].embedding

In [11]:
#임베딩 진행
policy_data['policy_embeddings'] = policy_data['정책'].apply(lambda x: get_embedding(x))
words_data['word_embeddings'] = words_data['용어'].apply(lambda x: get_embedding(x))

In [12]:
policy_data.head(3)

,rnum,정책,policy_embeddings
0,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...,"[-0.024120546877384186, 0.03090936876833439, 0..."
1,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...,"[-0.009894113056361675, 0.03836297243833542, 0..."
2,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...,"[-0.02647486887872219, 0.03164590895175934, 0...."


In [13]:
words_data.head(3)

,rnum,용어,word_embeddings
0,1,디지털 드로잉은 컴퓨터나 태블릿과 같은 디지털 기기를 사용하여 그림을 그리는 것을 ...,"[0.006628303322941065, -0.0026317525189369917,..."
1,2,만 나이는 사람이 실제로 살아온 정확한 연수를 기준으로 계산하는 나이입니다. 한국에...,"[0.015981122851371765, 0.021755356341600418, -..."
2,3,"비영리단체는 사회적, 교육적, 자선적, 문화적 또는 과학적 목적을 추구하기 위해 설...","[0.023582614958286285, 0.027814337983727455, 0..."


In [14]:
# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme':'http'}], http_auth=('elastic', es_pw) )

C:\Users\yh\AppData\Local\Temp\ipykernel_22864\3582269626.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme':'http'}], http_auth=('elastic', 'BD_pl-B5*iWsE+Jwwoqr') )


In [26]:
#정책 인덱스 없으면 생성, 청크x
index_name1 = 'policy'
if not es.indices.exists(index=index_name1):
    es.indices.create(index=index_name1)

# 데이터 인덱싱
def generate_docs(data):
    for idx, row in data.iterrows():
        yield {
            "_index": index_name1,
            "_id": idx,
            "_source": {
                "text": row['정책'],
                "embedding": row['policy_embeddings']
            }
        }

helpers.bulk(es, generate_docs(policy_data))

(555, [])

In [17]:
#용어 인덱스 없으면 생성, 청크x
index_name2 = 'word'
if not es.indices.exists(index=index_name2):
    es.indices.create(index=index_name2)

# 데이터 인덱싱
def generate_docs(data):
    for idx, row in data.iterrows():
        yield {
            "_index": index_name2,
            "_id": row['rnum'],
            "_source": {
                "text": row['용어'],
                "embedding": row['word_embeddings']
            }
        }

helpers.bulk(es, generate_docs(words_data))

(43, [])

In [ ]:
#===============================검색 테스트============================================

In [27]:
# 검색 쿼리 생성 함수
def policy_search(query, top_k=3):
    # 쿼리 텍스트 임베딩
    query_embedding = get_embedding(query)

    # 검색 쿼리
    search_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_embedding}
                }
            }
        }
    }

    # 검색 실행
    response = es.search(index='policy', body=search_query)
    return response['hits']['hits']


In [30]:
# 예제 검색
query = "청년 평생 교육 프로그램 다담다의 지원 내용은?"
results = policy_search(query)
for result in results:
    print(result['_source']['text'], result['_score'])
    print('==============================================================')

지역: 노원구
분야: 일자리
정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영(노원구)
정책 설명: '나를 담다', '우리를 담다', '미래를 담다', '여가를 담다'의 4개 영역, 총 30개 프로그램 운영
지원 내용:
- '나를 담다' (180명): 자신을 더 깊이 이해하고, 자신만의 브랜드 구축
  - 퍼스널 컬러와 스타일
  - 진로·적성검사
  - 심리검사
  - 퍼스널 브랜딩
  - 그림책 테라피
  - 원예 테라피
  - 다이어리 꾸미기
- '우리를 담다' (140명): 타인의 마음을 이해하는 심리검사와 소통하는 법을 배우는 프로그램
- '미래를 담다’ (355명): 청년이 미래를 대비할 수 있는 지식과 기술 제공
  - 자산관리방법
  - 경제교육 및 재테크 방법
  - 주거관리
  - N잡러(다중 직업) 배우기
  - 법률 교육
  - ChatGPT 활용
  - 메타버스 체험
  - 명사 토크콘서트
- '여가를 담다’ (170명): 청년의 다양한 취미를 즐기며 의미있는 여가시간 만들기
  - 기초 풋살 강의
  - 뜨개질
  - 디지털 드로잉
  - 식물 재배
  - 미술 이야기
  - 세계여행
주관 기관: 노원구 청년정책과
지원(신청) 기간: 2024-07-22~2024-11-30 (2024.07.22.(월) 부터 신청 가능)
지원 인원: 800명 1.5656385
지역: 영등포구
분야: 일자리
정책명: 청년도전 지원사업
정책 설명: 구직단념청년을 위한 프로그램으로, 다양한 체험을 통해 구직 의욕을 높이고 자신감을 강화합니다. 단기, 중기, 장기 모듈 수업과 참여수당을 제공합니다.
지원 조건:
- 구직단념청년: 신청일 이전 6개월 이상 취업·교육·직업훈련 미참여, 구직단념청년 문답표 21점 이상, 만 18~34세
- 자립준비청년: 아동복지시설 보호 후 퇴소 5년 이내 또는 퇴소 연장자, 만 18~34세
- 청소년복지시설 입·퇴소 청년: 청소년복지시설에서 6개월 이상 보호받은 만 18~34세
- 북한이탈청년: 북한을 이탈한 만 18~34세
- 

In [23]:
# 검색 쿼리 생성 함수
def word_search(query, top_k=3):
    # 쿼리 텍스트 임베딩
    query_embedding = get_embedding(query)

    # 검색 쿼리
    search_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_embedding}
                }
            }
        }
    }

    # 검색 실행
    response = es.search(index='word', body=search_query)
    return response['hits']['hits']


In [24]:
# 예제 검색
query = "구정이 무슨뜻이야?"
results = word_search(query)
for result in results:
    print(result['_source']['text'], result['_score'])
    print('==============================================================')

구정은 특정 지방 자치 단체인 구(區)의 행정을 의미합니다. 대한민국 같은 국가에서는 대도시를 여러 구로 나누어 각 구마다 자체적인 행정 기관을 두고 있습니다. 구정은 그 지역 내의 행정, 서비스 제공, 정책 시행, 지역 개발 및 관리 등 다양한 공공 업무를 관리합니다. 강남구청 같은 구청은 구민의 복지 향상과 지역 발전을 목표로 다양한 프로그램과 정책을 운영하며, 이러한 활동 전체가 구정 활동에 해당합니다. 구정은 구민의 일상생활과 밀접한 관련이 있으며, 구민의 삶의 질을 높이는 데 중요한 역할을 합니다. 1.5320867
조례(條例)는 지방자치단체의 의회에서 제정하는 법률로, 해당 지방자치단체 내에서만 적용되는 규범입니다. 중앙 정부가 제정하는 법률과는 달리, 조례는 지방자치단체가 자체적으로 필요한 사항을 규정하기 위해 만들며, 주민의 생활에 직접적인 영향을 미치는 규칙들을 포함합니다. 예를 들어, 주차 규정, 환경 보호, 지역 축제 운영 등이 조례에 의해 규정될 수 있습니다.

조례는 해당 지역 주민의 요구와 특성을 반영하여 제정되기 때문에, 지역 사회의 구체적인 상황에 맞는 규범을 제공하는 역할을 합니다. 1.3026613
통풍은 혈액 내 요산 농도가 높아져 관절에 요산 결정이 쌓여 염증과 통증을 유발하는 질환입니다. 주로 발가락 관절에서 시작되며, 급성 통증과 부기를 동반할 수 있습니다. 식이 요법과 약물 치료로 관리할 수 있습니다. 1.2600875
